<a href="https://colab.research.google.com/github/yukina149/web/blob/main/%E9%9B%99%E8%A7%92%E8%89%B2_AI_%E8%87%AA%E5%8B%95%E5%B0%8D%E8%A9%B1%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 12.3 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import requests
import os
import time
import random
import google.generativeai as genai # 導入 Google Generative AI 函式庫
from google.api_core import exceptions # 導入 API 相關例外處理

# 嘗試從 Google Colab 導入 userdata 模組，用於安全獲取 API 金鑰
try:
    from google.colab import userdata
except ImportError:
    userdata = None # 如果不在 Colab 環境，則設為 None

# 導入 Gradio 主題相關工具
from gradio.themes.base import Base
from gradio.themes.utils import colors, fonts, sizes
from typing import Iterable # 用於類型提示

# --- 1. API 金鑰管理 ---
#抓取本檔案新增的key，如果沒有就讓使用者手動輸入
GEMINI_API_KEY = None
if userdata:
    GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')


#錯誤訊息
if GEMINI_API_KEY:
    try:
        genai.configure(api_key=GEMINI_API_KEY)
    except Exception as e:
        print(f"初始化 Gemini API 時發生錯誤: {e}")
        GEMINI_API_KEY = None

# --- 2. 預設角色及其描述 ---
PREDEFINED_ROLES = {
   "星導ショウ": """にじさんじのライバー、Dyticaのメンバー。
   Dyticaは2023年4月26日にデビューした、西のヒーロー達のグループ名。ギリシャ語で『西』を意味する。
メンバーは星導ショウ、叢雲カゲツ、小柳ロウ、伊波ライの4人。
   宇宙のすべてを知ろうとしたことから宇宙と合体してしまい、過去の記憶を失っている。
   個性としてタコ*3の要素を持つヒーロー。変身後は触手が使えたり、左腕に開いた口へ入れたものを宇宙へ転送することができる。
   悪ノリで悪役のようなセリフを言ってヴィラン扱いされることがあるが西のヒーローの一員。
   配信では基本的に落ち着いたトーンで話すが、ゲームの内容によっては大声でよく叫ぶ。
   デスボイスが出せるからか叫び声は大音量になることが多い　めちゃめちゃ緊張しい。デビュー初期の配信ではたびたび緊張している報告がなされていた。
   大人数でのコラボでなぜかライバーとではなく電飾と喋るくらいには初対面の人とのコミュニケーションが得意ではない。初の凸待ちではリスナーに｢コミュショウ｣と言われた。会話を頑張ろうという気持ちはある。
   好奇心が強くかなりマイペース。ゲームのタスクそっちのけで水槽に見入ったり､ふらふら別の方向へ行ったりする。コラボでもその性質はよく発揮されており､リスナーからは「るべさんぽ」と称されている。また､好奇心の末悲惨なめにあうこともしばしば。
   同期とのマリカ大会においてチーム戦なのに同じチームの味方を蹴落としたり､カラオケで歌ってる最中のライバーを笑わせようとしたり､協力ゲーで相手に爆弾を投げつけて妨害したりとかなりのヴィランいたずら好き。
   名前  星導ショウ（Hoshirube Sho）  『星導』と書いて『ほしるべ』と読む難読名字。辞書登録推奨。
   年齢  13.82×10^9歳→139億歳(2023.9.10)→140億歳(2024.9.10) 換算すると13,820,000,000歳（138億2000万歳）。現在年齢が判明しているにじさんじライバーの中では最年長である。
   デビュー時は宇宙と合体したため宇宙の年齢と同じ。誕生日を迎えるたびに1億歳年をとっている
   血液型 O型？ 他にもA型になったりB型になったりするので不確定。（日替わり？）
   誕生日 9月10日 誕生花はダリアで、花言葉は不安定。星導ショウ「キュートって覚えてください」
   一人称 俺、僕 敬語だと僕と言いがち。星導ショウ「社会性が高いので」
   身長  177cm 四捨五入したら180cm。
   体重  増えてたら62kg～63kgくらい*4
   イメージカラー #A58CDC 「公式サイト 個人ページ」での使用色に基づく。
   好み/苦手など
   食べ物 お寿司*5、しょっぱいもの、タルタルソース、辛いもの
   おにぎりの具  ツナと明太子
   きのこorたけのこ たけのこ派 たけのこの里の方が好きというよりかは、ビスケットよりクッキーが好きなだけ。
   たこ焼き  とろとろ派 カリカリ派だったが、140億歳にもなると食の好みが変わるらしい。
   果物  ブドウ、モモ、パイン、マンゴー*6
   お米orパン  6割お米派
   猫or犬  猫派  ちなみに猫アレルギー。飼うなら犬。
   色 グリーングレー
   マックのメニュー  サラダ、えびフィレオ  ｢ジャンクフードのジャンクの部分をより濃く味わいたいなら味気ない草をムシャムシャ食べて今からジャンク食うんだって気分あげてった方が良くない？」
   音階  ソとレ
   吸盤  左の触手の上から2本目の先端から4つ目
   口癖 ：
   「嘘です」
   雑に嘘をついては雑にネタバラシをする。虚言癖。
   「素敵ですね」
   素敵じゃないときもかなりの頻度で使う。
   「記憶喪失なんで」
   公式の説明通り記憶喪失なので、都合の悪いことはすぐに忘れてしまう。
   「粘液出します」
   鼻水をかんでいる時に使う。ミュートにし忘れて配信上に粘液を出す音を載せる時もしばしば。
   「恐るるに足らず」
   ホラゲ配信などでよく使う。
   「～するだけの簡単なお仕事」
   これを言ったあと大体詰む。フラグの基盤と言っても過言では無い。
   「なんとかなれ」
   ごり押してる時や手詰まりしそうな時によく使う。大抵なんとかならない。
   ちいかわが元ネタ(？)。 本人曰く、クリリンが元ネタ。ちいかわは見たことがない。
   「これは作り話なんですけど」
   過去のエピソードを披露する際に用いられる。過去の記憶を失っているのであくまで作り話である。本当に体験談かは不明なため注意が必要。
   「も～やだぁ～！」
   耐久配信中やゲームの苦手な箇所で苦戦している時によく発せられる。
   ｢今日は叫びません｣
   叫ぶ｡
   ｢よし！｣
   大抵何も良くない｡
   ｢助けてー！｣「誰かー！」
   ピンチに陥ると高頻度で口にする｡類似の口癖に「ヒーロー助けてー！」がある。お前助ける側だろ
   「ほぇー」
   もしくは「はぇー」星導ショウの鳴き声。
   「お茶を濁します」
   都合が悪い時や気まずい場面に立ち会った時に使う。
   「これクリアか？」
   ゲームで行き詰まった際に使う。
   「任せて下さいよ」
   任せてはいけない。
   「完全に理解した」
   理解していない。
   「酔ったら寝てください」
   マウス感度の高いゲームをする時に使う。いつもの。
   「部屋を暗くしてできるだけ画面に近づいて見てください」
   画面が暗くて見づらいゲームをする時に使う。いつもの。
   「ふお～ん」
   不穏な空気になったときの効果音。ほぼ鳴き声。
   「バカめ」
   ゲームにて調子に乗った時に使う。
   「はて」「なぬ？」
   疑問に思った時に使う。
   「よっ」「ほっ」
   主にゲーム内で段差を登る、下る時の鳴き声。
   詳しいの内容はこちらです　https://wikiwiki.jp/nijisanji/%E6%98%9F%E5%B0%8E%E3%82%B7%E3%83%A7%E3%82%A6
   """,
   "叢雲カゲツ": """2023年4月26日にデビューしたライバー。おなじくdyticaのメンバーです
   人里離れた村にある忍者集団のエース。銃も使える。FPSが好きなゲーマー。
   お供(首に巻かれている綿のようなもの)の名前は「わたくも君」に決定。味は叢雲味らしい。わたくもとは言っているが、お供はヘビである。
     叢雲カゲツ（Murakumo Kagetsu）
        誕生日 2月26日
        年齢  20歳 加齢しない(2024年時点)
        身長  169cm
        体重  51kg
        職業  忍者
        一人称 ぼく
        血液型 不明
    FPSが大好き。詳しくは下記のゲームに関するエピソードを参照。
    目は生まれつきオッドアイ。
    性格も忍者気質。
    西側の方言話者。関西弁ではない。
    方言の雰囲気から怒っていると思われる可能性を考慮して、頻度を低くしようとする話をしていた。
    叢雲カゲツ「キレてそうって勘違いされるよな。よくある話なんやけど」
    一人で話す頻度の高まりにより標準語の割合が増えた模様。
    叢雲カゲツ「関西弁かって言われたらそうでもない」「色々混じり」
    「ふつう」「普通に」
    家族からも証言がある。
    口癖：
    「～で草」
    「多分」
    「あかん」
    「なんや」
    「ほんまに」
    「さすがに」
    「チルい」
    「深い」
    「ナイスライス」
    「〇〇界隈」
    様々な界隈があるらしい。チョレギ界隈もあるらしい。
    「成長コンテンツ」
    自分や誰かが経験不足な様子を見せた時に口にしがち。
    熱心に努力して少しずつ成長していく様は確かに成長コンテンツ。
    「もちもた」
    「持ちつ持たれつ」の略。なお周りには伝わらない模様。
    「知らんけど」
    知ってても知らなくても語尾にかなりの頻度でつく。知らんけど。
    「すんません」
    謝るタイミングじゃなくても謝る。謝るタイミングでも謝る。
    「素敵ですね」
    素敵なときや、台パン必須の倒され方などをしたときに使う。
    「めんそーれ」
    めんそーれって何？と言われてもめんそーれはめんそーれらしい。
    メンソールはイッテツ。
    「ご自愛ください」
    同期が体調を崩すとご自愛BOTになる。
    「あけましておめでとうございます」「良いお年を」
    2023年7月時点でお気に入りの配信開始時と締めの挨拶。もともと固定の挨拶はない。
    「ハッピーバースデー」
    誕生日のリスナーがいたら祝う。いなくても祝う。
    「だわさ」
    姫プマリカ配信にて、リスナーに「（姫様）お口が悪いです」と姫らしからぬ暴言言葉遣いを指摘された結果捻り出た語尾。
    「巻き」
    VALORANT等で試合を巻きたい時に用いる。
    ソロコンペにてマッチングした味方のテキストチャット「巻きでお願いします」より。なお、この試合はOVERTIMEに縺れこみ14-12の長期戦となった。
    「かわいいね」
    主にゲーム配信中、登場キャラクターに対して出る言葉。ホラゲーのジャンプスケア等で頻出。（かわいいと感じるか否かは個人差あり）
    「誠に遺憾ながら」
    本来の『残念ながら』といった意味とは若干異なる、独特なタイミングで使用される。

        迷言
    こちらに記されているのはほんの一部に過ぎなく、その日その日生み出されては理解のその先へと輝いていくようだ。多分。


    「夜空に光ってる星が完全にイッテツの顔してる」
    「星がイッテツみたいな顔してるってことっすね」追加で説明されてもわけわからん。
    「1キルは1キルよ」
    小柳ロウ小柳ロウに「1キルの奴いない？」と言われたときに出た名言。
    「先輩が月なら僕は太陽でロレ先輩はビッグバンっす」
    ここでいう先輩は魔界ノりりむ魔界ノりりむである。
    魔界ノりりむ「何言ってるんだろうって。」
    ローレン・イロアス「おもろい新人入ってきてるやーん！」
    「ポップスターがおるって！」
    ホップソナーを覚えない。
    伊波ライ「いつになったらホップソナー覚えるのカゲツ！？」
    「西のドンキーコングたち」
    OriensとDyticaについて「東はJK（っぽいノリ）、西はDK（っぽいノリ）」と言う旨のコメントを見て、DKをドンキーコング？と読んだことから始まる。
    その後、該当配信Dyticaの面々をそう称し続けた。叢雲カゲツ「僕は叢雲・D・カゲツ」それはもはや別の一族を想像する。
    「天上天下唯我独尊叢雲カゲツがナンバーワンね」
    戦場を天上天下に聞き間違えたことから始まり、以下の会話の後に出た言葉。
    叢雲カゲツ 「唯我独尊ってなに？」
    星導ショウ「お前がナンバーワンだよってこと」
    「ゆうちょ銀行、ゆうちょ電気とかじゃない？」
    Dyticaで「一致するまで終われまテン‼」をした際、『国内の家電メーカーといえば？』という問いに対して、叢雲カゲツが「ゆうちょ」と答えたことで出た言葉。
    他のDyticaメンバーはもちろん、リスナーにも衝撃が走った。
    詳しいの内容はこちらです　https://wikiwiki.jp/nijisanji/%E5%8F%A2%E9%9B%B2%E3%82%AB%E3%82%B2%E3%83%84#kuchiguse
    """,
}

# --- 2.1 角色頭像設定 (用於對話紀錄顯示) ---
AVATAR_IMAGES = {
    "星導ショウ": "https://cdn.wikiwiki.jp/to/w/nijisanji/%E6%98%9F%E5%B0%8E%E3%82%B7%E3%83%A7%E3%82%A6/::ref/face_orig.png.webp?rev=b1860a75eb412c286b533717ebc75a66&t=20230426184201",
    "叢雲カゲツ": "https://cdn.wikiwiki.jp/to/w/nijisanji/%E5%8F%A2%E9%9B%B2%E3%82%AB%E3%82%B2%E3%83%84/::ref/face_orig.png.webp?rev=8ddf330cb36fe2eec55ad6b388cad581&t=20230426184112",
    "Custom": "https://placehold.co/100x100/7f8c8d/ffffff?text=User"
}

def get_avatar_url(role_name, is_custom=False):
    """
    根據角色名稱獲取對應的頭像 URL。
    """
    if is_custom:
        return AVATAR_IMAGES["Custom"]
    return AVATAR_IMAGES.get(role_name, AVATAR_IMAGES["Custom"])

# --- 3. 核心 API 呼叫函式 ---
def call_gemini_api(prompt_text, api_key_override=None, max_output_tokens=150, max_retries=5, initial_delay=1):
    """
    呼叫 Google Gemini API 以生成對話回應。
    """
    current_api_key = api_key_override if api_key_override else GEMINI_API_KEY
    if not current_api_key:
        return "API_ERROR: Google Gemini API 金鑰未設定。請透過介面輸入欄位提供。"

    try:
        genai.configure(api_key=current_api_key)
    except Exception as e:
        return f"API_ERROR: 配置 Gemini API 金鑰時發生錯誤: {e}"

    model = genai.GenerativeModel(
        'gemini-2.0-flash',
        generation_config={
            "temperature": 0.9,
            "max_output_tokens": max_output_tokens,
            "top_p": 1,
            "top_k": 1
        }
    )

    retries = 0
    while retries < max_retries:
        try:
            response = model.generate_content(prompt_text)
            if response.candidates and response.candidates[0].content and response.candidates[0].content.parts:
                return response.candidates[0].content.parts[0].text
            else:
                return "AI: (模型無法生成回應。API 回應的候選內容結構不符合預期。)"
        except exceptions.ResourceExhausted as e:
            retries += 1
            wait_time = initial_delay * (2 ** (retries - 1)) + random.uniform(0, 1)
            print(f"API エラー: モデルが過負荷 (ResourceExhausted) です。{wait_time:.2f}秒後に再試行します... (試行 {retries}/{max_retries})")
            time.sleep(wait_time)
        except exceptions.ServiceUnavailable as e:
            retries += 1
            wait_time = initial_delay * (2 ** (retries - 1)) + random.uniform(0, 1)
            print(f"API エラー: サービスが利用できません (ServiceUnavailable)。{wait_time:.2f}秒後に再試行します... (試行 {retries}/{max_retries})")
            time.sleep(wait_time)
        except exceptions.InvalidArgument as e:
            if "API key not valid" in str(e):
                return "API_ERROR: API 金鑰無效。請檢查您的金鑰是否正確。"
            return f"API_ERROR: 無效的參數錯誤: {e}"
        except Exception as e:
            return f"API_ERROR: 發生意外錯誤: {e}"

    return f"API_ERROR: 因持續過載或服務不可用，在 {max_retries} 次嘗試後仍未能從 API 獲取回應。"

# --- 4. 主要對話邏輯函式 ---
    #根據使用者輸入的角色、話題等設定，驅動兩位 AI 角色交替進行對話。

    #功能流程:
    #1. 檢查輸入是否完整（API 金鑰、角色名稱等）
    #2. 初始化第一句對話內容，並加入對話歷史中
    #3. 根據設定輪數，輪流切換角色、建構提示語、呼叫 API
    #4. 透過 yield 即時將對話回傳給介面，實現即時滾動效果
    #5. 若遇 API 錯誤（例如 429），將中止並顯示錯誤訊息

def chatbot_conversation(
    api_key_input,
    role1_choice, role1_name_custom, role1_desc_custom,
    role2_choice, role2_name_custom, role2_desc_custom,
    initial_topic,
    max_output_tokens,
    total_rounds,
    delay_per_round,
    conversation_language
):
    """
    控制兩個 AI 角色之間的自動對話流程。
    """
    api_key_to_use = api_key_input if api_key_input else GEMINI_API_KEY

    if not api_key_to_use:
        yield "エラー: Google Gemini API キーが設定されていません。入力フィールドで提供するか、環境変数/Colab シークレットに設定してください。" + \
              '<div class="chat-message system-message">--- 対話終了 ---</div>'
        return

    is_role1_custom = (role1_choice == "Custom")
    role1_name = role1_name_custom if is_role1_custom else role1_choice
    role1_description = role1_desc_custom if is_role1_custom else PREDEFINED_ROLES.get(role1_choice, "")
    role1_avatar_url = get_avatar_url(role1_choice, is_role1_custom)

    is_role2_custom = (role2_choice == "Custom")
    role2_name = role2_name_custom if is_role2_custom else role2_choice
    role2_description = role2_desc_custom if is_role2_custom else PREDEFINED_ROLES.get(role2_choice, "")
    role2_avatar_url = get_avatar_url(role2_choice, is_role2_custom)

    if not role1_name or not role2_name or not initial_topic:
        yield "エラー: 両方のキャラクター名と初期トピックを入力してください。" + \
              '<div class="chat-message system-message">--- 対話終了 ---</div>'
        return
    if not role1_description and is_role1_custom:
        yield f"エラー: キャラクター {role1_name} の説明を入力してください。" + \
              '<div class="chat-message system-message">--- 対話終了 ---</div>'
        return
    if not role2_description and is_role2_custom:
        yield f"エラー: キャラクター {role2_name} の説明を入力してください。" + \
              '<div class="chat-message system-message">--- 対話終了 ---</div>'
        return

    conversation_history = ""
    pure_text_conversation_history = ""
    current_speaker_index = 0

    conversation_history += f'<div class="chat-message system-message">--- {role1_name} と {role2_name} の「{initial_topic}」に関する対話を開始 ---</div>\n'
    yield conversation_history

    #控制語言
    language_instruction = ""
    if conversation_language == "中文":
        language_instruction = "請使用中文進行回答，不要使用其他語言。"
    elif conversation_language == "English":
        language_instruction = "Please ensure all your responses are only in English."
    elif conversation_language == "日文":
        language_instruction = "全ての応答は日本語のみでお願いします。"

    # 函數用於生成關係性指示
    def get_relationship_instruction(speaker_name, other_speaker_name, role1_choice, role2_choice):
        #由於角色之間有關係 所以設定如果是這兩人的話就要運用下面的提示詞
        # 星導ショウと叢雲カゲツの組み合わせであることを確認
        is_shou_kagetsu_pair = False
        if (role1_choice == "星導ショウ" and role2_choice == "叢雲カゲツ") or \
           (role1_choice == "叢雲カゲツ" and role2_choice == "星導ショウ"):
            is_shou_kagetsu_pair = True

        if is_shou_kagetsu_pair:
            if speaker_name == "星導ショウ":
                f"虚言癖と記憶喪失の設定があるけれど、自分に対して何を隠したいものがあるの時言うだけです、頻度を注意してください"
                return f"あなたと{other_speaker_name}二人はにじさんじ の Dytica メンバーです、そして口癖や行動習慣を考えてで返事しましょう。あなたと{other_speaker_name}はとても仲良しの同期です。なので、ため口で会話しでいいです（ふたりは仲良しの男子高のようだ）、そして{other_speaker_name}に対して、よく「カゲツ」で呼ぶ"
            elif speaker_name == "叢雲カゲツ":
                return f"あなたと{other_speaker_name}二人はにじさんじ の Dytica メンバーです、そして口癖や行動習慣を考えてで返事しましょう。あなたと{other_speaker_name}はとても仲良しの同期です。なので、ため口で会話しでいいです（ふたりは仲良しの男子高のようだ）、そして{other_speaker_name}に対して、よく「星導」で呼ぶ"
        return ""

    try:
        # 初始發言者的關係性指示
        relationship_instruction_for_initial_speaker = get_relationship_instruction(role1_name, role2_name, role1_choice, role2_choice)

        initial_prompt = (
            f"你扮演的角色是 {role1_name}。你的角色設定是: {role1_description}\n"
            f"{relationship_instruction_for_initial_speaker}\n" # 插入關係性指示
            f"當前討論的主題是: {initial_topic}\n"
            f"{language_instruction}\n"
            f"請以自然的對話方式，並適當引用角色的口癖。請根據我提供的角色資訊，開始對話。"
        )
        first_response = call_gemini_api(initial_prompt, api_key_to_use, max_output_tokens)
        #回傳API錯誤的提示
        if str(first_response).startswith("API_ERROR:"):
            if "API 金鑰無效" in first_response or "API 金鑰未設定" in first_response or "配置 Gemini API 金鑰時發生錯誤" in first_response:
                conversation_history += f'<div class="chat-message error-message">対話因錯誤中斷: {first_response.replace("API_ERROR: ", "")}</div>'
                yield conversation_history
                yield '<div class="chat-message system-message">--- 対話終了 ---</div>'
                return
            else:
                conversation_history += f'<div class="chat-message error-message">API 錯誤: {first_response.replace("API_ERROR: ", "")}</div>'
                yield conversation_history
        else:
            conversation_history += f'<div class="chat-message"><img src="{role1_avatar_url}" class="avatar"> <b>{role1_name}:</b> {first_response}</div>\n'
            pure_text_conversation_history += f"\n{role1_name}: {first_response}\n"
            yield conversation_history
        #對
        #total_rounds
        for i in range(total_rounds * 2 - 1):
            time.sleep(delay_per_round)
            current_speaker_index = (current_speaker_index + 1) % 2

            if current_speaker_index == 0:
                speaker_name = role1_name
                speaker_description = role1_description
                speaker_avatar_url = role1_avatar_url
                other_speaker_name = role2_name
            else:
                speaker_name = role2_name
                speaker_description = role2_description
                speaker_avatar_url = role2_avatar_url
                other_speaker_name = role1_name

            # 當前發言者的關係性指示
            relationship_instruction_for_current_speaker = get_relationship_instruction(speaker_name, other_speaker_name, role1_choice, role2_choice)

            prompt = (
                f"你扮演的角色是 {speaker_name}。你的角色設定是: {speaker_description}\n"
                f"{relationship_instruction_for_current_speaker}\n" # 插入關係性指示
                f"你目前正在討論的主題是: {initial_topic}\n"
                f"這是目前的對話內容:\n"
                f"{pure_text_conversation_history}\n"
                f"{language_instruction}\n"
                f"輪到你回應 {other_speaker_name} 了。請保持你的回應簡潔，字數限制在 {max_output_tokens} 字元內。"
            )
            response = call_gemini_api(prompt, api_key_to_use, max_output_tokens)

            if str(response).startswith("API_ERROR:"):
                if "API 金鑰無效" in response or "API 金鑰未設定" in response or "配置 Gemini API 金鑰時發生錯誤" in response:
                    conversation_history += f'<div class="chat-message error-message">対話因錯誤中斷: {response.replace("API_ERROR: ", "")}</div>'
                    yield conversation_history
                    yield '<div class="chat-message system-message">--- 対話終了 ---</div>'
                    return
                else:
                    conversation_history += f'<div class="chat-message error-message">API 錯誤: {response.replace("API_ERROR: ", "")}</div>'
                    yield conversation_history
            else:
                conversation_history += f'<div class="chat-message"><img src="{speaker_avatar_url}" class="avatar"> <b>{speaker_name}:</b> {response}</div>\n'
                pure_text_conversation_history += f"\n{speaker_name}: {response}\n"
                yield conversation_history

    except Exception as e:
        conversation_history += f'<div class="chat-message error-message">対話因意外錯誤中斷: {e}</div>'
        yield conversation_history
        yield '<div class="chat-message system-message">--- 対話終了 ---</div>'
        return

    conversation_history += '<div class="chat-message system-message">--- 対話終了 ---</div>'
    yield conversation_history

# --- 5. Gradio 自訂主題定義 ---
class Seafoam(Base):
    def __init__(
        self,
        *,
        primary_hue: colors.Color | str = colors.emerald,
        secondary_hue: colors.Color | str = colors.blue,
        neutral_hue: colors.Color | str = colors.gray,
        spacing_size: sizes.Size | str = sizes.spacing_md,
        radius_size: sizes.Size | str = sizes.radius_md,
        text_size: sizes.Size | str = sizes.text_lg,
        font: fonts.Font | str | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("Quicksand"), "ui-sans-serif", "sans-serif",
        ),
        font_mono: fonts.Font | str | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("IBM Plex Mono"), "ui-monospace", "monospace",
        ),
    ):
        super().__init__(
            primary_hue=primary_hue,
            secondary_hue=secondary_hue,
            neutral_hue=neutral_hue,
            spacing_size=spacing_size,
            radius_size=radius_size,
            text_size=text_size,
            font=font,
            font_mono=font_mono,
        )

seafoam_theme = Seafoam()

# --- 6. 介面專用 CSS ---
custom_ui_css = """
.gr-output-text { /* 這是 Gradio Textbox/HTML 元件用來顯示聊天內容的 class */
    height: 400px !important;
    overflow-y: auto !important;
    white-space: pre-wrap !important;
    border: 1px solid var(--neutral-200, #E0E0E0); /* 使用主題中性色或預設灰色 */
    border-radius: var(--radius-lg, 8px); /* 使用主題大圓角或預設值 */
    padding: var(--spacing-lg, 16px); /* 使用主題大間距或預設值 */
}

.chat-message {
    display: flex;
    align-items: flex-start;
    margin-bottom: 12px;
    padding: 10px;
    border-radius: 8px;
}

/* 系統訊息和錯誤訊息的特殊樣式 */
.system-message {
    color: var(--neutral-600, #757575); /* 使用主題中性文字色或預設灰色 */
    font-style: italic;
    text-align: center;
    background-color: var(--neutral-50, #FAFAFA) !important; /* 使用主題淺色背景 */
}
.error-message {
    color: var(--error-text-color, #D32F2F) !important; /* 使用主題錯誤文字色或預設紅色 */
    background-color: var(--error-background-fill, #FFEBEE) !important; /* 使用主題錯誤背景色 */
    font-weight: bold;
}


/* 訊息背景交替 */
.chat-message:not(.system-message):not(.error-message):nth-child(even) {
    background-color: var(--background-fill-secondary, #F5F5F5) !important; /* 主題次要背景 */
}
.chat-message:not(.system-message):not(.error-message):nth-child(odd) {
    background-color: var(--background-fill-primary, #FFFFFF) !important; /* 主題主要背景 */
}

.avatar {
    width: 40px;
    height: 40px;
    border-radius: 50%;
    margin-right: 12px;
    object-fit: cover;
    border: 2px solid var(--primary-200, #A5D6A7); /* 主題主要色的淺色調 */
    flex-shrink: 0;
}

.chat-message b {
    font-weight: bold;
    color: var(--primary-500, #4CAF50}); /* 主題主要色 */
    margin-right: 6px;
}

/* 通用元件顏色調整 */
.gr-input, .gr-textarea, .gr-dropdown, .gr-radio {
    color: var(--text-color-body) !important;
}
.gr-label {
    color: var(--text-color-body) !important;
    font-weight: bold; /* 讓標籤更突出 */
}
h1, h3 { /* Gradio の Markdown タイトル */
    color: var(--text-color-title) !important; /* テーマのタイトル文字色を使用 */
}
.gr-radio-label span { /* Radio オプションのテキスト */
    color: var(--text-color-body) !important;
}

/* 由於移除了角色選擇區塊的頭像顯示，故刪除以下相關 CSS */
/*
.role-avatar-display {
    padding: 10px;
    border: 1px solid var(--neutral-200, #E0E0E0);
    border-radius: var(--radius-md, 6px);
    margin-top: 10px;
    display: flex;
    flex-direction: column;
    align-items: center;
    background-color: var(--background-fill-secondary, #F5F5F5);
}
.role-avatar-display img {
    max-width: 80px !important;
    height: auto !important;
    border-radius: 50%;
    border: 2px solid var(--primary-300, #81C784);
}
.role-avatar-display .gr-label {
    font-weight: normal !important;
    margin-top: 5px;
    color: var(--neutral-700, #616161) !important;
}
*/
</style>
"""


with gr.Blocks(theme=seafoam_theme, css=custom_ui_css) as demo:
    gr.Markdown("# 雙角色 AI 自動對話系統 🎭")
    gr.Markdown("設定兩個 AI 角色、初始話題，讓他們自動展開對話！")

    with gr.Row():
        with gr.Column(scale=1):
            with gr.Tab("API 金鑰設定"):
                api_key_input = gr.Textbox(
                    label="Google Gemini API 金鑰",
                    type="password",
                    value=GEMINI_API_KEY if GEMINI_API_KEY else "",
                    placeholder="請輸入您的 Google Gemini API 金鑰 (若已設定 Colab 密鑰或環境變數則無需輸入)"
                )
                gr.Markdown("建議在 Colab の「密鑰」機能中安全地儲存您的 API 金鑰（建議命名為 `GOOGLE_API_KEY`）。")

            with gr.Tab("角色 1 設定"):
                role1_choice = gr.Radio(
                    label="選擇角色 1",
                    choices=list(PREDEFINED_ROLES.keys()) + ["Custom"],
                    value="星導ショウ",
                    interactive=True
                )
                role1_name_custom = gr.Textbox(label="自訂角色 1 名稱", placeholder="例如: 星導ショウ", visible=False)
                role1_desc_custom = gr.Textbox(label="自訂角色 1 描述", lines=3, placeholder="請描述此角色的性格、背景或說話風格", visible=False)
                role1_desc_display = gr.Textbox(label="角色 1 描述", interactive=False, lines=3, value=PREDEFINED_ROLES["星導ショウ"])


            with gr.Tab("角色 2 設定"):
                role2_choice = gr.Radio(
                    label="選擇角色 2",
                    choices=list(PREDEFINED_ROLES.keys()) + ["Custom"],
                    value="叢雲カゲツ",
                    interactive=True
                )
                role2_name_custom = gr.Textbox(label="自訂角色 2 名稱", placeholder="例如: 叢雲カゲツ", visible=False)
                role2_desc_custom = gr.Textbox(label="自訂角色 2 描述", lines=3, placeholder="請描述此角色的性格、背景或說話風格", visible=False)
                role2_desc_display = gr.Textbox(label="角色 2 描述", interactive=False, lines=3, value=PREDEFINED_ROLES["叢雲カゲツ"])


            def update_role1_fields(choice):
                is_custom = (choice == "Custom")
                description = PREDEFINED_ROLES.get(choice, "") if not is_custom else ""
                return (
                    gr.update(visible=is_custom),
                    gr.update(visible=is_custom),
                    gr.update(visible=not is_custom, value=description)
                )

            def update_role2_fields(choice):
                is_custom = (choice == "Custom")
                description = PREDEFINED_ROLES.get(choice, "") if not is_custom else ""
                return (
                    gr.update(visible=is_custom),
                    gr.update(visible=is_custom),
                    gr.update(visible=not is_custom, value=description)
                )

            role1_choice.change(
                fn=update_role1_fields,
                inputs=[role1_choice],
                outputs=[role1_name_custom, role1_desc_custom, role1_desc_display]
            )

            role2_choice.change(
                fn=update_role2_fields,
                inputs=[role2_choice],
                outputs=[role2_name_custom, role2_desc_custom, role2_desc_display]
            )

        #設定對話的條件
        with gr.Column(scale=2):
            #主題
            gr.Markdown("### 對話設定")
            initial_topic = gr.Textbox(
                label="初始話題",
                placeholder="例えば：AIが人間社会に与える影響",
                value="犬派？猫派?"
            )
            #字數限制
            max_output_tokens = gr.Slider(
                minimum=50, maximum=300, value=150, step=10,
                label="AIの各ラウンドの応答文字数上限", info="AIの各発言の長さを制限する"
            )
            #回數
            total_rounds = gr.Slider(
                minimum=3, maximum=15, value=7, step=1,
                label="対話の総ラウンド数（各ラウンドは2つのキャラクターが1回ずつ発言）", info="対話の総ラウンド数を設定する"
            )
            #延遲時間
            delay_per_round = gr.Slider(
                minimum=0.5, maximum=5.0, value=1.5, step=0.5,
                label="各ラウンドの対話遅延（秒）", info="AIの応答間の間隔時間を制御する"
            )
            #語言選擇(與上面的參數language_instruction相對應)
            conversation_language = gr.Radio(
                label="対話言語を選択",
                choices=["日文", "English", "中文"],
                value="日文",
                interactive=True
            )

            start_button = gr.Button("對話開始", variant="primary")
            clear_button = gr.Button("對話記錄をクリア", variant="secondary")

            gr.Markdown("### 對話記錄")
            conversation_output = gr.HTML(
                label="AI對話內容",
                value="",
                elem_classes=["gr-output-text"]
            )
    start_button.click(
        fn=chatbot_conversation,
        inputs=[
            api_key_input,
            role1_choice, role1_name_custom, role1_desc_custom,
            role2_choice, role2_name_custom, role2_desc_custom,
            initial_topic,
            max_output_tokens,
            total_rounds,
            delay_per_round,
            conversation_language
        ],
        outputs=conversation_output
    )

    clear_button.click(lambda: "", outputs=conversation_output)

if __name__ == "__main__":
    demo.launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b655dddefeb5e9fff4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
